In [40]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain.agents import tool

@tool
def add(a: int, b: int) -> int:
    """Adds a and b"""
    return int(a) + int(b)


@tool
def subtract(a: int, b: int) ->int:
    """Subtracts b from a"""
    return int(a) - int(b)

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b"""
    return int(a) * int(b)

print(multiply.name)
print(multiply.description)
print(multiply.args)

tools = [add, subtract, multiply]

print(multiply)

multiply
Multiplies a and b
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
name='multiply' description='Multiplies a and b' args_schema=<class 'langchain_core.utils.pydantic.multiply'> func=<function multiply at 0x000001BF8C013D80>


In [23]:
from pydantic import BaseModel, Field

class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")
    

@tool("search-tool", args_schema=SearchInput)
def search(q: str) -> str:
    """Lookup things online"""
    return "LangChain"

print(search)
print(search.args)

name='search-tool' description='Lookup things online' args_schema=<class '__main__.SearchInput'> func=<function search at 0x0000018472E1E200>
{'query': {'description': 'should be a search query', 'title': 'Query', 'type': 'string'}}


In [20]:
def mymultiply(a: int, b: int) -> int:
    """Multiplies a and b"""
    return int(a) * int(b)


class MultiplyInput(BaseModel):
    """Multiply two integers"""
    a: int = Field(..., description="First Number")
    b: int = Field(..., description="Second Number")



In [ ]:
from langchain.tools import StructuredTool

mytool = StructuredTool.from_function(
    name="custom-multiply",
    func=mymultiply,
    args_schema=MultiplyInput,
    description="Multiply 2 numbers"
)

mytool

StructuredTool(name='custom-multiply', description='Multiply 2 numbers', args_schema=<class '__main__.MultiplyInput'>, func=<function mymultiply at 0x00000184742EA200>)

In [28]:
from langchain_community.tools import TavilySearchResults

tavily_search_tool = TavilySearchResults(max_results=5)

search_results = tavily_search_tool.invoke({"query": "Give summary on H1B Restrictions issued by Trump"})

search_results

[{'title': 'Understanding the New $100000 H-1B Fee and its Effect on U.S. ...',
  'url': 'https://www.employmentlawworldview.com/understanding-the-new-100000-h-1b-fee-and-its-effect-on-u-s-employers/',
  'content': 'On Friday, September 19, 2025, President Trump issued a Proclamation entitled “Restrictions on Entry of Certain Nonimmigrant Workers” that imposes a $100,000 fee for most new H-1B visa petitions and restricts the ability of certain H-1B visa holders to enter the United States.The H-1B visa is the workhorse of the U.S. immigration system, currently used by approximately three-quarters of a million U.S. workers to provide high-skilled labor in “specialty occupations” requiring at least a related',
  'score': 0.8029425},
 {'title': 'H-1B FAQ - USCIS',
  'url': 'https://www.uscis.gov/newsroom/alerts/h-1b-faq',
  'content': 'On Friday, Sept. 19, 2025, President Donald J. Trump signed a Proclamation, "Restriction on Entry of Certain Nonimmigrant Workers," that took an important, 

In [42]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

llm = ChatOpenAI(model="gpt-4o-mini")

tools = [tavily_search_tool, add, subtract, multiply]

llm_with_tools = llm.bind_tools(tools)

response = llm_with_tools.invoke([HumanMessage(content="Hi")])

print(f"Content String : {response.content}")
print(f"Tool Calls: {response.tool_calls}")

Content String : Hello! How can I assist you today?
Tool Calls: []


In [41]:
response = llm_with_tools.invoke([HumanMessage(content="Tell me about Trump H1B New Policy (search for latest 2025 updates)")])

print(f"Content String : {response.content}")
print(f"Tool Calls: {response.tool_calls}")

Content String : 
Tool Calls: [{'name': 'tavily_search_results_json', 'args': {'query': 'Trump H1B New Policy 2025 updates'}, 'id': 'call_aLeGekKGgJ5bXJU49c8SUheW', 'type': 'tool_call'}]


In [ ]:
from typing import List
from langchain.tools import Tool

def find_tool_by_name(tools: List[Tool], tool_name: str) -> Tool:
    for tool in tools:
        if tool.name == tool_name:
            return tool
    
    raise ValueError(f"Tool with name {tool_name} not found")

In [48]:
template = """
    Answer the following questions as best you can. You have access to the following tools:

    {tools}

    You have to use these tools only, even if you can compute the response directly

    Use the following format:

    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question

    Begin!

    Question: {input}
    Thought: {agent_scratchpad}
"""

In [50]:
from langchain.tools.render import render_text_description
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(template=template).partial(
    tools = render_text_description(tools),
    tool_names = ", ".join([tool.name for tool in tools])
)


prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': 'tavily_search_results_json - A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.\nadd(a: int, b: int) -> int - Adds a and b\nsubtract(a: int, b: int) -> int - Subtracts b from a\nmultiply(a: int, b: int) -> int - Multiplies a and b', 'tool_names': 'tavily_search_results_json, add, subtract, multiply'}, template='\n    Answer the following questions as best you can. You have access to the following tools:\n\n    {tools}\n\n    You have to use these tools only, even if you can compute the response directly\n\n    Use the following format:\n\n    Question: the input question you must answer\n    Thought: you should always think about what to do\n    Action: the action to take, should be one of [{tool_names}]\n    Action Input: the input to the action\n    Observa

In [51]:
llm = ChatOpenAI(
    model = "gpt-4o-mini",
    stop_sequences=["Observation", "\nObservation"]
)

In [54]:
from langchain.agents.output_parsers import ReActSingleInputOutputParser

intermediate_steps = []

chain = {
            "input": lambda x: x["input"],
            "agent_scratchpad": lambda x: x["intermediate_steps"]
        } |prompt | llm | ReActSingleInputOutputParser()


agent_action = chain.invoke({
    "input": "What is 2 + 3 multiplied by 4",
    "intermediate_steps": intermediate_steps
})

agent_action

AgentAction(tool='multiply', tool_input='3, 4  \n', log='To solve the expression \\(2 + 3 \\times 4\\), I need to remember the order of operations (PEMDAS/BODMAS), which prioritizes multiplication over addition.\n\nFirst, I will calculate \\(3 \\times 4\\) and then add 2 to the result.\n\nAction: multiply  \nAction Input: 3, 4  \n')

In [ ]:
intermediate_steps.append((agent_action, 12))


[(AgentAction(tool='multiply', tool_input='3, 4  \n', log='To solve the expression \\(2 + 3 \\times 4\\), I need to remember the order of operations (PEMDAS/BODMAS), which prioritizes multiplication over addition.\n\nFirst, I will calculate \\(3 \\times 4\\) and then add 2 to the result.\n\nAction: multiply  \nAction Input: 3, 4  \n'),
  12)]

In [59]:
from langchain.agents.format_scratchpad import  format_log_to_str

format_log_to_str(intermediate_steps=intermediate_steps)

'To solve the expression \\(2 + 3 \\times 4\\), I need to remember the order of operations (PEMDAS/BODMAS), which prioritizes multiplication over addition.\n\nFirst, I will calculate \\(3 \\times 4\\) and then add 2 to the result.\n\nAction: multiply  \nAction Input: 3, 4  \n\nObservation: 12\nThought: '

In [63]:
intermediate_steps = []

agent = {
            "input": lambda x: x["input"],
            "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"])
        } |prompt | llm | ReActSingleInputOutputParser()


agent_action = agent.invoke({
    "input": "What is 2 + 3 multiplied by 4",
    "intermediate_steps": intermediate_steps
})
agent_action

AgentAction(tool='multiply', tool_input='3, 4  \n', log='To solve the expression \\(2 + 3 \\times 4\\), I need to follow the order of operations, where multiplication comes before addition. First, I will multiply 3 by 4, and then I will add 2 to the result.\n\nAction: multiply  \nAction Input: 3, 4  \n')

In [64]:
intermediate_steps.append((agent_action, 12))

agent_action = agent.invoke({
    "input": "What is 2 + 3 multiplied by 4",
    "intermediate_steps": intermediate_steps
})
agent_action

AgentAction(tool='add', tool_input='2, 12  \n\n', log='Now that I have multiplied 3 by 4 to get 12, I need to add 2 to this result.\n\nAction: add  \nAction Input: 2, 12  \n\n')

In [65]:
intermediate_steps.append((agent_action, 14))

agent_action = agent.invoke({
    "input": "What is 2 + 3 multiplied by 4",
    "intermediate_steps": intermediate_steps
})
agent_action

AgentFinish(return_values={'output': '14'}, log='I now know the final answer  \nFinal Answer: 14')

In [9]:
from langchain.schema import AgentAction, AgentFinish
from typing import List
from langchain.tools import Tool
from langchain.tools.render import render_text_description
from langchain.prompts import PromptTemplate
from langchain_community.tools import TavilySearchResults
from langchain_openai import ChatOpenAI


def find_tool_by_name(tools: List[Tool], tool_name: str) -> Tool:
    for tool in tools:
        if tool.name == tool_name:
            return tool
    
    raise ValueError(f"Tool with name {tool_name} not found")


template = """
    Answer the following questions as best you can. You have access to the following tools:

    {tools}

    You have to use these tools only, even if you can compute the response directly

    Use the following format:

    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question

    Begin!

    Question: {input}
    Thought: {agent_scratchpad}
"""
from langchain_community.tools import TavilySearchResults

tavily_search_tool = TavilySearchResults(max_results=5)

tools = [tavily_search_tool, add, subtract, multiply]

prompt = PromptTemplate.from_template(template=template).partial(
    tools = render_text_description(tools),
    tool_names = ", ".join([tool.name for tool in tools])
)

llm = ChatOpenAI(
    model = "gpt-4o-mini",
    stop_sequences=["Observation", "\nObservation"]
)


In [10]:
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.agents.format_scratchpad import  format_log_to_str

intermediate_steps = []

agent = {
            "input": lambda x: x["input"],
            "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"])
        } |prompt | llm | ReActSingleInputOutputParser()


In [13]:
from typing import Union
agent_step = ""

while not isinstance(agent_step, AgentFinish):
    agent_step: Union[AgentAction, AgentFinish] = agent.invoke({
        "input": "tell me about Sivaprasad valluru",
        "intermediate_steps": intermediate_steps
    })
    
    print(f"Agent Step = {agent_step}")
    if(isinstance(agent_step, AgentAction)):
        tool_name = agent_step.tool
        tool_to_use = find_tool_by_name(tools, tool_name)
        tool_input = agent_step.tool_input
        inputs = tool_input.split(",")
        if(len(inputs) == 1):
            inputs = inputs[0]
        
        print(f"Tool Selected = {tool_to_use}")
        print(f"Tool Inputs = {inputs}")
        observation = tool_to_use.invoke(inputs)
        print(f"Observation = {observation}")
        
        
        intermediate_steps.append((agent_step, str(observation)))
        print("Intermediate Step = {format_log_to_str(intermediate_steps)}")
    
    if(isinstance(agent_step, AgentFinish)):
        print(agent_step.return_values)

Agent Step = tool='tavily_search_results_json' tool_input='Sivaprasad Valluru"  \n' log='I need to gather information about Sivaprasad Valluru. To do this, I will perform a search for his name to find reliable and comprehensive details.  \nAction: tavily_search_results_json  \nAction Input: "Sivaprasad Valluru"  \n'
Tool Selected = api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********'))
Tool Inputs = Sivaprasad Valluru"  

Observation = [{'title': 'SivaPrasad - Learning why what when', 'url': 'https://www.learnagenticai.in/home/about-us#:~:text=SivaPrasad Valluru is a instructor,done contribution to Spring framework.', 'content': 'SivaPrasad Valluru is a instructor with 20 years of experince in IT . He has worked with Motorola, Alcatel Lucent and TechMahindra earlier. Now, he delivers corporate trainings. He has done contribution to Spring framework.', 'score': 0.9854}, {'title': 'SivaPrasad Valluru is a instructor with 20 years of ... - Instagram', 'url': 'https://w

In [17]:
from typing import Union
agent_step = ""

while not isinstance(agent_step, AgentFinish):
    agent_step: Union[AgentAction, AgentFinish] = agent.invoke({
        "input": "What is 2 + 3 multiplied by 4",
        "intermediate_steps": intermediate_steps
    })
    
    print(f"Agent Step = {agent_step}")
    if(isinstance(agent_step, AgentAction)):
        tool_name = agent_step.tool
        tool_to_use = find_tool_by_name(tools, tool_name)
        tool_input = agent_step.tool_input
        inputs = tool_input.split(",")
        if(len(inputs) == 1):
            inputs = inputs[0]
        else:
            inputs = tool_input.replace("\n", "").replace("'", "").strip()
        
        print(f"Tool Selected = {tool_to_use}")
        print(f"Tool Inputs = {inputs}")
        observation = tool_to_use.invoke(inputs)
        print(f"Observation = {observation}")
        
        
        intermediate_steps.append((agent_step, str(observation)))
        print("Intermediate Step = {intermediate_steps}")
    
    if(isinstance(agent_step, AgentFinish)):
        print(agent_step.return_values)

Agent Step = tool='multiply' tool_input='3, 4  \n\n' log="To solve the expression \\(2 + 3 \\times 4\\), I need to perform the multiplication first, as per the order of operations (PEMDAS/BODMAS).\n\nLet's calculate the multiplication part first.\n\nAction: multiply  \nAction Input: 3, 4  \n\n"
Tool Selected = name='multiply' description='Multiplies a and b' args_schema=<class 'langchain_core.utils.pydantic.multiply'> func=<function multiply at 0x0000021BFCC9DEE0>
Tool Inputs = 3, 4


ValidationError: 2 validation errors for multiply
a
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='3, 4', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing
b
  Field required [type=missing, input_value={'a': '3, 4'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

In [1]:
from langsmith import Client
client = Client()
prompt = client.pull_prompt("hwchase17/react", include_model=True)

prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [6]:
from langchain.agents import create_react_agent,AgentExecutor
from langchain_openai import ChatOpenAI
from langchain_community.tools import TavilySearchResults

tavily_search_tool = TavilySearchResults(max_results=5)

llm = ChatOpenAI(
    model="gpt-4o-mini",
    # stop_sequences= ["Observation", "\nObservation"]
)


tools = [tavily_search_tool, add, subtract, multiply]
agent = create_react_agent(llm=llm, 
                           tools=tools, 
                           prompt=prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
result = agent_executor.invoke({"input": "Tell me about Leela Prasad Jagu"})




> Entering new AgentExecutor chain...
I need to find current information about Leela Prasad Jagu, as I don't have any knowledge about this individual. 
Action: tavily_search_results_json
Action Input: "Leela Prasad Jagu"[{'title': 'Leela Prasad - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/Leela_Prasad', 'content': 'Leela Prasad is an Indian historian based in the United States. A scholar of Indian history and the anthropology of ethics, she is a 2023 Guggenheim Fellow and is St. Purandar Das Distinguished Professor of Religious Studies at Brown University.(\n\nBiography\n\n[edit] [...] As an academic, Prasad specialises in the anthropology of ethics, early Indic philology, history of India, and oral history.( She curated a Historical Society of Pennsylvania exhibition on the history of the South Asian Americans in the Delaware Valley, and she subsequently edited a tie-in 1999 volume _Live Like the Banyan Tree_.( In 2006, she worked on two books: as co-editor of _Gender and Sto

In [24]:
from pydantic import BaseModel
from langchain.tools import StructuredTool

class ArthimeticArgs(BaseModel):
    a: int
    b: int
    

def add(a: int, b: int) -> int:
    """Adds a and b"""
    return int(a) + int(b)

def subtract(a: int, b: int) ->int:
    """Subtracts b from a"""
    return int(a) - int(b)

def multiply(a: int, b: int) -> int:
    """Multiplies a and b"""
    print("Args:: ", a, b)
    return int(a) * int(b)


multiply_tool = StructuredTool.from_function(
    func=multiply,
    args_schema=ArthimeticArgs,
    description="Multiplies 2 numbers"
)

add_tool = StructuredTool.from_function(
    func=add,
    args_schema=ArthimeticArgs,
    description="Adds 2 numbers"
)

subtract_tool = StructuredTool.from_function(
    func=subtract,
    args_schema=ArthimeticArgs,
    description="Subtracts 2 numbers"
)

tools = [tavily_search_tool, add_tool, subtract_tool, multiply_tool]

from langsmith import Client
from langchain.prompts import PromptTemplate
client = Client()
base_prompt = client.pull_prompt("hwchase17/react", include_model=True)

custom_instructions = """
When calling tools, ALWAYS pass arguments as strict JSON not as a string. 
- Use integers, not strings.
- Example: pass arguments like 3, 4 
- NEVER combine numbers into one string like "3, 4".
- NEVER use tuples like (3, 4).
"""

prompt = PromptTemplate(
    input_variables=base_prompt.input_variables,
    template=base_prompt.template + "\n\n" + custom_instructions
)


In [25]:
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

result = agent_executor.invoke({"input": "What is 2 + 3 multiplied by 4"})



> Entering new AgentExecutor chain...
Question: What is 2 + 3 multiplied by 4  
Thought: I need to follow the order of operations (PEMDAS/BODMAS). First, I need to calculate multiplication before addition. So, I will multiply 3 by 4 and then add 2 to the result.  
Action: multiply  
Action Input: 3, 4  

ValidationError: 2 validation errors for ArthimeticArgs
a
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='3, 4', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing
b
  Field required [type=missing, input_value={'a': '3, 4'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

In [ ]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an AI.
                You should not use tools parallely
                You should not make your own conclusions even if it is easy, instead use appropriate tools"""),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, max_iterations=5)
result = agent_executor.invoke({"input": "Tell me about Elon Musk"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Elon Musk'}`


[{'title': 'Elon Musk | SpaceX, Tesla, Twitter, X, Trump, DOGE, & Facts', 'url': 'https://www.britannica.com/money/Elon-Musk', 'content': 'Elon Musk (born June 28, 1971, Pretoria, South Africa) is a South African–born American entrepreneur who cofounded the electronic payment firm PayPal and formed SpaceX, maker of launch vehicles and spacecraft. He was also one of the first significant investors in, as well as chief executive officer of, the electric car manufacturer Tesla. In addition, Musk acquired Twitter (later X) in 2022. Musk led the Department of Government Efficiency (DOGE) in U.S. President Donald Trump’s second [...] Elon Musk cofounded the electronic payment firm PayPal, and in 2002 he founded SpaceX, a company that makes rockets and spacecraft. He was a major early funder of Tesla, which makes electric cars and batteries, and became its chief executive officer in